# Keras基本例

初期設定だとTheanoが背後で動くこととなる.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using Theano backend.


## 使用データ

別途構築したMarketDataを用いる.



In [44]:
inputCSV = pd.read_csv('InputDNNMarket.csv')
inputCSV.head(10)

,Unnamed: 0,JP/US,JGB10,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close,JP/US.Lag,JGB10.Lag,S&P/Close.Lag,...,S&P/AdjClose.MA30,VIX/Close.MA30,JP/US.MA120,JGB10.MA120,S&P/Close.MA120,S&P/Volume.MA120,S&P/AdjClose.MA120,VIX/Close.MA120,Nikkei/Close.Lag_1,Ans
0,2001-06-20,123.30,1.201,1223.140015,1.350100e+09,1223.140015,12674.64,0.002684,0.029135,0.008709,...,1261.507666,13461.619000,120.279083,1.321554,1254.843961,1.200070e+09,1254.843961,13386.141000,0.022706,2
1,2001-06-21,124.40,1.182,1237.040039,1.546820e+09,1237.040039,12962.43,0.008921,-0.015820,0.011364,...,1260.902999,13426.440333,120.353500,1.317746,1254.041461,1.195202e+09,1254.041461,13380.065500,0.006340,0
2,2001-06-22,124.25,1.171,1225.349976,1.189200e+09,1225.349976,13044.61,-0.001206,-0.009306,-0.009450,...,1260.225663,13393.130000,120.420667,1.313921,1253.433128,1.193188e+09,1253.433128,13373.207167,-0.011356,1
3,2001-06-25,124.02,1.144,1218.599976,1.050100e+09,1218.599976,12896.47,-0.001851,-0.023057,-0.005509,...,1259.214994,13360.578333,120.487750,1.310142,1252.789294,1.192644e+09,1252.789294,13366.185583,0.006385,0
4,2001-06-26,123.82,1.140,1216.760010,1.198900e+09,1216.760010,12978.82,-0.001613,-0.003497,-0.001510,...,1258.125663,13324.738000,120.547583,1.306600,1252.088961,1.192707e+09,1252.088961,13360.921500,-0.011545,1
5,2001-06-27,124.25,1.133,1211.069946,1.162100e+09,1211.069946,12828.98,0.003473,-0.006140,-0.004676,...,1255.661662,13295.895000,120.614167,1.302867,1251.237293,1.191587e+09,1251.237293,13355.890917,-0.011622,1
6,2001-06-28,124.63,1.125,1226.199951,1.327300e+09,1226.199951,12679.88,0.003058,-0.007061,0.012493,...,1253.585327,13254.868667,120.673083,1.299583,1250.398794,1.190888e+09,1250.398794,13351.547667,0.022805,2
7,2001-06-29,124.73,1.183,1224.380005,1.832360e+09,1224.380005,12969.05,0.000802,0.051556,-0.001484,...,1251.332662,13224.578000,120.731167,1.297108,1249.614043,1.195524e+09,1249.614043,13348.391917,-0.016799,1
8,2001-07-02,124.22,1.263,1236.719971,1.128300e+09,1236.719971,12751.18,-0.004089,0.067625,0.010079,...,1248.795663,13177.056333,120.787000,1.295142,1248.898376,1.194586e+09,1248.898376,13342.099833,0.005194,0
9,2001-07-03,124.48,1.272,1234.449951,6.221100e+08,1234.449951,12817.41,0.002093,0.007126,-0.001836,...,1246.297994,13134.597000,120.847000,1.292925,1248.130042,1.189723e+09,1248.130042,13335.707833,-0.014698,1


In [45]:
# データの並びをシャッフル -> 時系列関係無い場合
# 今回は他の手法との対比からシャッフルしないことにする
calcData = inputCSV # .reindex(np.random.permutation(inputCSV.index))
calcData.head(10)

,Unnamed: 0,JP/US,JGB10,S&P/Close,S&P/Volume,S&P/AdjClose,VIX/Close,JP/US.Lag,JGB10.Lag,S&P/Close.Lag,...,S&P/AdjClose.MA30,VIX/Close.MA30,JP/US.MA120,JGB10.MA120,S&P/Close.MA120,S&P/Volume.MA120,S&P/AdjClose.MA120,VIX/Close.MA120,Nikkei/Close.Lag_1,Ans
0,2001-06-20,123.30,1.201,1223.140015,1.350100e+09,1223.140015,12674.64,0.002684,0.029135,0.008709,...,1261.507666,13461.619000,120.279083,1.321554,1254.843961,1.200070e+09,1254.843961,13386.141000,0.022706,2
1,2001-06-21,124.40,1.182,1237.040039,1.546820e+09,1237.040039,12962.43,0.008921,-0.015820,0.011364,...,1260.902999,13426.440333,120.353500,1.317746,1254.041461,1.195202e+09,1254.041461,13380.065500,0.006340,0
2,2001-06-22,124.25,1.171,1225.349976,1.189200e+09,1225.349976,13044.61,-0.001206,-0.009306,-0.009450,...,1260.225663,13393.130000,120.420667,1.313921,1253.433128,1.193188e+09,1253.433128,13373.207167,-0.011356,1
3,2001-06-25,124.02,1.144,1218.599976,1.050100e+09,1218.599976,12896.47,-0.001851,-0.023057,-0.005509,...,1259.214994,13360.578333,120.487750,1.310142,1252.789294,1.192644e+09,1252.789294,13366.185583,0.006385,0
4,2001-06-26,123.82,1.140,1216.760010,1.198900e+09,1216.760010,12978.82,-0.001613,-0.003497,-0.001510,...,1258.125663,13324.738000,120.547583,1.306600,1252.088961,1.192707e+09,1252.088961,13360.921500,-0.011545,1
5,2001-06-27,124.25,1.133,1211.069946,1.162100e+09,1211.069946,12828.98,0.003473,-0.006140,-0.004676,...,1255.661662,13295.895000,120.614167,1.302867,1251.237293,1.191587e+09,1251.237293,13355.890917,-0.011622,1
6,2001-06-28,124.63,1.125,1226.199951,1.327300e+09,1226.199951,12679.88,0.003058,-0.007061,0.012493,...,1253.585327,13254.868667,120.673083,1.299583,1250.398794,1.190888e+09,1250.398794,13351.547667,0.022805,2
7,2001-06-29,124.73,1.183,1224.380005,1.832360e+09,1224.380005,12969.05,0.000802,0.051556,-0.001484,...,1251.332662,13224.578000,120.731167,1.297108,1249.614043,1.195524e+09,1249.614043,13348.391917,-0.016799,1
8,2001-07-02,124.22,1.263,1236.719971,1.128300e+09,1236.719971,12751.18,-0.004089,0.067625,0.010079,...,1248.795663,13177.056333,120.787000,1.295142,1248.898376,1.194586e+09,1248.898376,13342.099833,0.005194,0
9,2001-07-03,124.48,1.272,1234.449951,6.221100e+08,1234.449951,12817.41,0.002093,0.007126,-0.001836,...,1246.297994,13134.597000,120.847000,1.292925,1248.130042,1.189723e+09,1248.130042,13335.707833,-0.014698,1


In [46]:
# 入力データと教師データに分割
pd_y = calcData['Ans']
pd_x = calcData.drop(["Unnamed: 0", 'Nikkei/Close.Lag_1', "Ans"], axis=1)

# numpy配列にデータフレームから変換
x_array = pd_x.as_matrix()
y_array = pd_y.as_matrix()

In [47]:
# 全レコード数
pd_x.shape

(3912, 24)

In [48]:
# 正規化し,float32の型に変換
x_scale = scale(x_array).astype(np.float32)

# サンプルの一部をテストデータ用に分割
N = 3500 # 使用するtrainデータ,残りがテストデータになる
X_train, X_test = np.split(x_scale, [N])

/Users/kohno/.pyenv/versions/3.5.1/envs/devkeras/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


### 教師データのラベリング
変換が必要な点に注意.

In [49]:
batch_size = 500 # 学習のバッチサイズ
nb_classes = 3 # 分類数(0からラベル振る必要あり)
nb_epoch = 50 # エポック数

In [50]:
y_scale = y_array.astype(np.int32)
Y_train, Y_test = np.split(y_scale, [N])

# 離散の分類問題の場合、ベクトルのラベルをto_categoricalで変換しておく必要がある.
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

## Deep learningのモデル定義

以下では暫定的に多層ニューラルネットである中間層2層のDNNを用いる.

なお、活性化関数にReLUを用いる.
また途中に過学習を防ぎ汎化性能を上げる手法であるdropoutが仕込んである.

In [51]:
model = Sequential()
model.add(Dense(100, input_shape=(24,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

### モデルは一度コンパイルする形式を取るタイプ

サマリーが出てちょっと見やすい.

In [52]:
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_17 (Dense)                 (None, 100)           2500        dense_input_6[0][0]              
____________________________________________________________________________________________________
activation_17 (Activation)       (None, 100)           0           dense_17[0][0]                   
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 100)           0           activation_17[0][0]              
____________________________________________________________________________________________________
dense_18 (Dense)                 (None, 100)           10100       dropout_12[0][0]                 
___________________________________________________________________________________________

### 学習と結果

fitで学習し,evaluateでtestデータから汎化性能をチェックする.

In [53]:
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 3500 samples, validate on 412 samples
Epoch 1/50
3500/3500 [==============================] - 0s - loss: 1.0775 - acc: 0.4917 - val_loss: 0.9220 - val_acc: 0.6311
Epoch 2/50
3500/3500 [==============================] - 0s - loss: 1.0127 - acc: 0.5614 - val_loss: 0.9167 - val_acc: 0.6311
Epoch 3/50
3500/3500 [==============================] - 0s - loss: 0.9797 - acc: 0.5829 - val_loss: 0.8945 - val_acc: 0.6311
Epoch 4/50
3500/3500 [==============================] - 0s - loss: 0.9622 - acc: 0.5877 - val_loss: 0.8838 - val_acc: 0.6311
Epoch 5/50
3500/3500 [==============================] - 0s - loss: 0.9526 - acc: 0.5923 - val_loss: 0.8794 - val_acc: 0.6311
Epoch 6/50
3500/3500 [==============================] - 0s - loss: 0.9396 - acc: 0.5991 - val_loss: 0.8670 - val_acc: 0.6311
Epoch 7/50
3500/3500 [==============================] - 0s - loss: 0.9235 - acc: 0.5977 - val_loss: 0.8576 - val_acc: 0.6335
Epoch 8/50
3500/3500 [==============================] - 0s - loss: 0.9246 - ac

だいたい65%の的中率、うーん微妙